In [59]:
from __future__ import division
import pandas as pd
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression as lr
from sklearn.linear_model import LinearRegression as linr
from multiprocessing import Pool, cpu_count

In [60]:
cols = ['index','year', 'accidentsAndIncidents', 'deviceStatus', 'obstruction', 'precipitation', 'roadwork', 'trafficConditions', 'trial']
eventTypes = ['accidentsAndIncidents', 'deviceStatus', 'obstruction', 'precipitation', 'roadwork', 'trafficConditions']

In [61]:
events = pd.read_csv("train.csv", sep=",", header=None)
events.columns= cols

In [143]:
print(events.iloc[len(events) - 1]['accidentsAndIncidents'])

295.0


In [135]:
#del events['index']
del events['trial']

In [64]:
def find_int(row):
    return int(row['trial'].split('_')[1])
events['id'] = events.apply(lambda row: find_int(row), axis = 1)

In [65]:
trials = events['id'].unique()

In [160]:
def find_predictions(trial_id):
    train_df = events[events.id == trial_id]
    pred = []
    #print(train_df)
    for event in eventTypes:
        #regression = np.polyfit(train_df['year'], train_df[event], 1)
        #pred.append(int(regression[0]*2014 + regression[1]))
        regr = linr(fit_intercept = True, normalize = True)
        x = train_df['year'].reshape(-1,1)
        y = train_df[event]
        regr.fit(x, y)
        r = regr.score(x, y)
        predict_val = regr.predict(2014)
        val13 = train_df.iloc[train_df.shape[0]-1][event]
        #if(r > .8):
            #avg = np.mean([predict_val, val13])
            #pred.append(int(avg))
        #else:
            #pred.append(val13)
        pred.append(int(np.mean([predict_val, val13])))
    wr.writerow(pred)

In [161]:
prediction = open('2014pred_avg.csv', 'a')
wr = csv.writer(prediction, dialect='excel')

count = 0
total = len(trials) + 1
for trial in range(1, 500):
    if(count %500 == 0):
        print("percent complete " + str((count/len(trials))*100) + " %")
        prediction.flush()
    find_predictions(trial)
    count += 1
prediction.flush()

percent complete 0.0 %
